In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../2_data_preparation/cleaned_data.csv')
clustered = pd.read_csv('data_clustered.csv')

In [3]:
clustered = clustered[['name', 'cluster']]
clustered

,name,cluster
0,Lerone Murphy,4
1,Edson Barboza,4
2,Khaos Williams,4
3,Carlston Harris,2
4,Themba Gorimbo,2
...,...,...
2391,Nick Sanzo,3
2392,Jim Mullen,3
2393,Yoshiki Takahashi,1
2394,Wallid Ismail,1


In [4]:
data['winner'].value_counts()

winner
f1    7289
0      111
Name: count, dtype: int64

In [5]:
# filter data where clear decision
data = data[data['winner'] == 'f1']
data['winner'].value_counts()

winner
f1    7289
Name: count, dtype: int64

In [6]:
data['method'].value_counts()

method
Decision - Unanimous    2645
KO/TKO                  2443
Submission              1435
Decision - Split         766
Name: count, dtype: int64

In [7]:
data = data[data['method'].isin(['Decision - Unanimous', 'Decision - Split'])]

In [8]:
data['method'].value_counts()

method
Decision - Unanimous    2645
Decision - Split         766
Name: count, dtype: int64

In [9]:

# Merge the cluster column from clustered DataFrame based on fighter names (f1 and f2)
data = pd.merge(data, clustered[['name', 'cluster']], left_on='f1', right_on='name', how='left')
data.rename(columns={'cluster': 'winner_cluster'}, inplace=True)

data = pd.merge(data, clustered[['name', 'cluster']], left_on='f2', right_on='name', how='left')
data.rename(columns={'cluster': 'loser_cluster'}, inplace=True)

In [10]:
odds_df = data[['winner_cluster', 'loser_cluster']]

In [11]:
odds_df.head(2)

,winner_cluster,loser_cluster
0,4,4
1,2,2


In [12]:
# filter data where clear decision
data = data[data['winner'] == 'f1']
data['winner'].value_counts()

winner
f1    3411
Name: count, dtype: int64

In [13]:
odds_df['winner_cluster'].value_counts()

winner_cluster
2    1669
4    1491
1     237
0       7
3       7
Name: count, dtype: int64

In [14]:
import matplotlib.cm as cm

# Group by winner and loser clusters to count the number of wins and losses
win_counts = odds_df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
loss_counts = odds_df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)

# Calculate win percentage by cluster matchup
win_percentage = (win_counts / (win_counts + loss_counts)) * 100
# Round win percentages to nearest integer
win_percentage_rounded = win_percentage.round()

# Display win percentages by cluster matchup
win_percentage_rounded.style.background_gradient(cmap='Blues', axis=None)

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,nan,50.000000,36.000000,nan,20.000000
1,50.000000,50.000000,23.000000,75.000000,25.000000
2,64.000000,77.000000,50.000000,100.000000,52.000000
3,nan,25.000000,0.000000,50.000000,nan
4,80.000000,75.000000,48.000000,nan,50.000000


In [15]:
(win_counts + loss_counts)

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,0,2,14,0,5
1,2,92,456,4,327
2,14,456,1626,9,941
3,0,4,9,12,0
4,5,327,941,0,1576


In [16]:
data.groupby(['weightclass', 'winner_cluster']).size().unstack(fill_value=0)

winner_cluster,0,1,2,3,4
weightclass,,,,,
bantamweight,0,18,184,0,241
featherweight,0,35,183,0,192
flyweight,1,14,112,0,183
heavyweight,0,26,96,3,84
light heavyweight,0,14,153,0,79
lightweight,1,39,338,3,231
middleweight,3,32,209,1,144
strawweight,0,12,49,0,135
welterweight,2,47,345,0,202


In [17]:
data['weightclass'].value_counts()

weightclass
lightweight          612
welterweight         596
bantamweight         443
featherweight        410
middleweight         389
flyweight            310
light heavyweight    246
heavyweight          209
strawweight          196
Name: count, dtype: int64

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to calculate win percentages for a given weight class
def calculate_win_percentage(df):
    win_counts = df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
    loss_counts = df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)
    win_percentage = (win_counts / (win_counts + loss_counts)) * 100
    return win_percentage.round().fillna(0)  # Round and fill NaN with 0

# List of unique weight classes
weight_classes = data['weightclass'].unique()

# Create a dictionary to store win percentages for each weight class
win_percentage_by_weight_class = {}

# Calculate win percentages for each weight class
for weight_class in weight_classes:
    df_filtered = data[data['weightclass'] == weight_class]
    win_percentage_by_weight_class[weight_class] = calculate_win_percentage(df_filtered)

# # Plot each heatmap individually
# for weight_class, win_percentage in win_percentage_by_weight_class.items():
#     plt.figure(figsize=(6, 4))
#     sns.heatmap(win_percentage, annot=True, cmap="Blues", fmt="g", cbar=True)
#     plt.title(f'Win Percentages for Weight Class: {weight_class}')
#     plt.xlabel('Loser Cluster')
#     plt.ylabel('Winner Cluster')
#     plt.show()


In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to calculate win percentages for a given weight class
def calculate_win_percentage(df):
    win_counts = df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
    loss_counts = df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)
    win_percentage = (win_counts / (win_counts + loss_counts)) * 100
    return win_percentage.round().fillna(0)  # Round and fill NaN with 0

# List of unique weight classes in the desired order
weight_classes_ordered = ['strawweight', 'flyweight', 'bantamweight', 'featherweight', 
                          'lightweight', 'welterweight', 'middleweight', 
                          'light heavyweight', 'heavyweight']

# Create a dictionary to store win percentages DataFrames for each weight class
win_percentage_df_by_weight_class = {}

# Calculate win percentages for each weight class in the specified order
for weight_class in weight_classes_ordered:
    df_filtered = data[data['weightclass'] == weight_class]
    win_percentage_df_by_weight_class[weight_class] = calculate_win_percentage(df_filtered)

# Display the win percentages as styled DataFrames with background gradient
for weight_class, win_percentage_df in win_percentage_df_by_weight_class.items():
    print(f'Win Percentages for Weight Class: {weight_class}')
    display(win_percentage_df.style.background_gradient(cmap='Blues', axis=None))
    print('\n' + '='*50 + '\n')


Win Percentages for Weight Class: strawweight


loser_cluster,0,1,2,4
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,33.000000,31.000000
2,0.000000,67.000000,50.000000,63.000000
4,0.000000,69.000000,37.000000,50.000000




Win Percentages for Weight Class: flyweight


,0,1,2,4
winner_cluster,,,,
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,50.000000,19.000000,17.000000
2,0.000000,81.000000,50.000000,47.000000
4,0.000000,83.000000,53.000000,50.000000




Win Percentages for Weight Class: bantamweight


...

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,0.000000,0.000000,50.000000,0.000000,0.000000
1,0.000000,50.000000,17.000000,0.000000,29.000000
2,50.000000,83.000000,50.000000,0.000000,50.000000
3,0.000000,0.000000,0.000000,50.000000,0.000000
4,0.000000,71.000000,50.000000,0.000000,50.000000




Win Percentages for Weight Class: light heavyweight


loser_cluster,1,2,4
winner_cluster,,,
1,50.000000,14.000000,32.000000
2,86.000000,50.000000,53.000000
4,68.000000,47.000000,50.000000




Win Percentages for Weight Class: heavyweight


loser_cluster,1,2,3,4
winner_cluster,,,,
1,50.000000,32.000000,100.000000,40.000000
2,68.000000,50.000000,100.000000,57.000000
3,0.000000,0.000000,50.000000,0.000000
4,60.000000,43.000000,0.000000,50.000000


> add to df to be able to visualize in tableau

In [25]:
import pandas as pd

# Assuming data is your original DataFrame
# Group by winner and loser clusters to count the number of wins and losses
win_counts = data.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
loss_counts = data.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)

# Calculate win percentage by cluster matchup
win_percentage = (win_counts / (win_counts + loss_counts)) * 100
# Round win percentages to nearest integer
win_percentage_rounded = win_percentage.round().fillna(0)  # Fill NaN with 0 for cases with no matches

# Reset index and melt the DataFrame to make it long-form
win_percentage_melted = win_percentage_rounded.reset_index().melt(id_vars='winner_cluster', var_name='loser_cluster', value_name='win_percentage')

# Merge the win percentage data with the original data
data_with_win_percentages = pd.merge(
    data, 
    win_percentage_melted, 
    how='left', 
    left_on=['winner_cluster', 'loser_cluster'], 
    right_on=['winner_cluster', 'loser_cluster']
)

# Check the merged DataFrame
display(data_with_win_percentages.head())

# Save the DataFrame to a CSV file for visualization in Tableau
data_with_win_percentages.to_csv('data_with_win_percentages.csv', index=False)


,end_time,f1,f2,fight_date,method,referee,round,time_format,weightclass,winner,f1_body,f1_clinch,f1_control,f1_distance,f1_dob,f1_ground,f1_head,f1_height,f1_kd,f1_leg,f1_reach,f1_record,f1_rev,f1_sapm,f1_sig_str,f1_sig_str_%,f1_slpm,f1_stance,f1_str__acc_,f1_str__def,f1_sub__avg_,f1_sub_att,f1_td,f1_td_%,f1_td_acc_,f1_td_avg_,f1_td_def_,f1_total_str,f1_weight,f2_body,f2_clinch,f2_control,f2_distance,f2_dob,f2_ground,f2_head,f2_height,f2_kd,f2_leg,f2_reach,f2_record,f2_rev,f2_sapm,f2_sig_str,f2_sig_str_%,f2_slpm,f2_stance,f2_str__acc_,f2_str__def,f2_sub__avg_,f2_sub_att,f2_td,f2_td_%,f2_td_acc_,f2_td_avg_,f2_td_def_,f2_total_str,f2_weight,gender,year,name_x,winner_cluster,name_y,loser_cluster,win_percentage
0,5:00,Lerone Murphy,Edson Barboza,2024-05-18,Decision - Unanimous,Herb Dean,5,5,featherweight,f1,27 of 34,13 of 17,5:19,205 of 344,"Jul 22, 1991",2 of 3,172 of 308,175.26,0,21 of 22,185.42,Record: 14-0-1,0,2.60,220 of 364,60%,5.01,Orthodox,54%,60%,0.6,0,4 of 6,66%,52%,1.59,46%,259 of 410,65.77,15 of 21,2 of 3,5:19,77 of 239,"Jan 21, 1986",0 of 0,38 of 189,180.34,0,26 of 32,190.50,Record: 24-12-0,0,4.65,79 of 242,32%,4.11,Orthodox,44%,56%,0.1,0,0 of 0,0,50%,0.45,72%,79 of 242,65.77,M,2024,Lerone Murphy,4,Edson Barboza,4,50.0
1,5:00,Themba Gorimbo,Ramiz Brahimaj,2024-05-18,Decision - Unanimous,Mark Smith,3,3,welterweight,f1,31 of 35,35 of 38,10:13,6 of 10,"Jan 23, 1991",2 of 4,9 of 13,185.42,0,3 of 4,195.58,Record: 13-4-0,1,1.08,43 of 52,82%,2.53,Orthodox,69%,46%,0.0,0,5 of 6,83%,58%,4.04,78%,110 of 134,77.11,3 of 4,3 of 4,10:13,0 of 2,"Nov 17, 1992",0 of 0,0 of 2,177.80,0,0 of 0,182.88,Record: 10-5-0,0,3.69,3 of 6,50%,1.74,Orthodox,41%,45%,1.6,0,1 of 8,12%,35%,1.61,44%,15 of 20,77.11,M,2024,Themba Gorimbo,2,Ramiz Brahimaj,2,50.0
2,5:00,Abus Magomedov,Warlley Alves,2024-05-18,Decision - Unanimous,Dan Miragliotta,3,3,middleweight,f1,7 of 11,2 of 2,11:56,9 of 27,"Sep 02, 1990",11 of 15,15 of 32,187.96,0,0 of 1,198.12,Record: 26-6-1,0,3.73,22 of 44,50%,2.88,Orthodox,44%,55%,0.0,0,6 of 7,85%,77%,2.65,100%,98 of 168,83.91,4 of 4,0 of 1,11:56,16 of 37,"Jan 04, 1991",0 of 0,7 of 26,180.34,0,5 of 8,182.88,Record: 15-8-0,0,3.95,16 of 38,42%,3.04,Orthodox,48%,50%,0.8,0,0 of 0,0,50%,1.22,62%,17 of 39,83.91,M,2024,Abus Magomedov,2,Warlley Alves,2,50.0
3,5:00,Alatengheili,Kleydson Rodrigues,2024-05-18,Decision - Unanimous,Herb Dean,3,3,bantamweight,f1,3 of 8,0 of 0,8:16,15 of 53,"Dec 14, 1991",3 of 6,14 of 50,165.10,0,1 of 1,167.64,Record: 17-9-2,0,4.67,18 of 59,30%,2.64,Orthodox,32%,57%,0.0,0,3 of 4,75%,45%,1.99,75%,107 of 183,61.23,10 of 15,0 of 0,8:16,37 of 88,"Oct 22, 1995",0 of 0,15 of 55,165.10,0,12 of 18,170.18,Record: 8-4-0,0,2.29,37 of 88,42%,4.58,Orthodox,57%,53%,0.6,0,0 of 1,0%,22%,0.60,53%,70 of 126,61.23,M,2024,Alatengheili,2,Kleydson Rodrigues,4,52.0
4,5:00,Vanessa Demopoulos,Emily Ducote,2024-05-18,Decision - Split,Chris Tognoni,3,3,strawweight,f1,18 of 25,3 of 4,0:00,106 of 289,"Sep 22, 1988",0 of 0,76 of 250,157.48,0,15 of 18,149.86,Record: 11-5-0,0,5.69,109 of 293,37%,3.98,Orthodox,41%,41%,0.6,0,0 of 3,0%,11%,0.42,30%,126 of 311,52.16,22 of 26,0 of 0,0:00,140 of 211,"Jan 01, 1994",3 of 3,86 of 150,157.48,0,35 of 38,160.02,Record: 13-9-0,0,7.84,143 of 214,66%,7.45,Orthodox,52%,57%,0.0,0,0 of 0,0,14%,0.20,94%,147 of 218,52.16,M,2024,Vanessa Demopoulos,1,Emily Ducote,4,25.0
